In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：AutoML 图像分割模型用于在线预测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_image_segmentation_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_image_segmentation_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示如何使用Python的Vertex客户端库创建图像分割模型，并使用Google Cloud的[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)进行在线预测。

### 数据集

本教程使用的数据集是[TODO](https://)。这个数据集不需要任何特征工程。您在本教程中将使用的数据集版本存储在一个公共云存储桶中。

### 目标

在本教程中，您将使用Vertex客户端库从Python脚本创建一个AutoML图像分割模型并部署以进行在线预测。您也可以使用`gcloud`命令行工具或在Google Cloud控制台上在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 将`Model`资源部署到一个服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`。

### 费用

本教程使用 Google Cloud（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 基于您预期的使用情况生成成本估算。

安装

安装最新版本的 Vertex 客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

重新启动内核

一旦您已安装了Vertex客户端库和Google *cloud-storage*，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU 运行时

*如果您的环境支持，确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的 Google Cloud 项目

**无论您使用的是哪种笔记本环境，都需要执行以下步骤。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。第一次创建帐户时，您将获得$300 的免费信用额用于支付计算/存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Google Cloud Notebook 中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保云 SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**: Jupyter 将以 `!` 为前缀的行视为 shell 命令，并将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，该变量用于本文档其余部分的操作。以下是Vertex支持的区域。我们建议您选择最靠近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不会使用多区域存储桶来进行Vertex训练。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行现场教程会话，您可能正在使用共享的测试账户或项目。为了避免在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并附加到在本教程中将要创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账号

**如果您正在使用谷歌云笔记本**，您的环境已经经过验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证以通过oAuth验证您的账户。

**否则**，请按照以下步骤进行操作：

在云控制台中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

在**为此服务帐号授予项目访问权限**部分，点击角色下拉列表。在过滤框中输入“Vertex”，选择**Vertex管理员**。在过滤框中输入“存储对象管理员”，选择**存储对象管理员**。

点击创建。包含您密钥的 JSON 文件将下载到本地环境。

在下面的单元格中输入您服务帐号密钥的路径作为 GOOGLE_APPLICATION_CREDENTIALS 变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库并定义常量

#### 导入 Vertex 客户端库

在我们的Python环境中导入Vertex客户端库。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、流水线和终端服务的顶点 API 服务终点。
- `PARENT`：顶点位置根路径，用于数据集、模型、作业、流水线和终端资源。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### AutoML常数

设置特定于AutoML数据集和训练的常数：

- 数据集模式：告诉“数据集”资源服务数据集的类型。
- 数据标签（注释）模式：告诉“数据集”资源服务数据的标签方式。
- 数据集训练模式：告诉“管道”资源服务为哪种任务（例如分类）训练模型。

In [ ]:
# Image Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml"
# Image Labeling type
LABEL_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/ioformat/image_segmentation_io_format_1.0.0.yaml"
# Image Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_segmentation_1.0.0.yaml"

# 教程

现在你准备好开始创建自己的AutoML图像分割模型了。

## 设置客户端

顶点客户端库以客户/服务器模型的形式工作。在您的一侧（Python脚本）中，您将创建一个客户端，向顶点服务器发送请求并接收响应。

在本教程中，您将针对工作流程中的不同步骤使用不同的客户端。因此，请提前设置它们。

- 数据集服务用于`Dataset`资源。
- 模型服务用于`Model`资源。
- 用于训练的Pipeline服务。
- 用于部署的Endpoint服务。
- 用于提供服务的Prediction服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

## 数据集

现在您的客户准备好了，训练模型的第一步是创建一个托管的数据集实例，然后将标记的数据上传到其中。

### 创建 `Dataset` 资源实例

使用辅助函数 `create_dataset` 来创建 `Dataset` 资源的实例。此函数执行以下操作：

1. 使用数据集客户端服务。
2. 创建一个 Vertex `Dataset` 资源（`aip.Dataset`），带有以下参数:
 - `display_name`: 您选择的可读名称。
 - `metadata_schema_uri`: 数据集类型的架构。
3. 调用客户端数据集服务方法 `create_dataset`，带有以下参数:
 - `parent`: 您的 `Database`, `Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
 - `dataset`: 您创建的 Vertex 数据集对象实例。
4. 该方法返回一个 `operation` 对象。

`operation` 对象是 Vertex 处理长时间运行操作的异步调用方式。虽然这个步骤通常很快，但在您的项目中第一次使用它时，由于需要进行配置，可能会有较长延迟。

您可以使用 `operation` 对象获取操作（例如创建 `Dataset` 资源）的状态，或者取消操作，通过调用操作方法:

| 方法      | 描述 |
| ----------- | ----------- |
| result()    | 等待操作完成并以 JSON 格式返回结果对象。      |
| running()   | 返回操作是否仍在运行的 True/False 值。        |
| done()      | 返回操作是否已完成的 True/False 值。 |
| canceled()  | 返回操作是否已取消的 True/False 值。 |
| cancel()    | 取消操作（可能需要最多 30 秒）。 |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        dataset = aip.Dataset(
            display_name=name, metadata_schema_uri=schema, labels=labels
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("unknown-" + TIMESTAMP, DATA_SCHEMA)

现在保存您创建的“数据集”资源实例的独特数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### 数据准备

图像的 Vertex `Dataset` 资源对您的数据有一些要求：

- 图像必须存储在 Cloud 存储桶中。
- 每个图像文件必须是图片格式（PNG、JPEG、BMP等）。
- 必须在您的 Cloud 存储桶中存储一个索引文件，该文件包含每个图像的路径和标签。
- 索引文件必须是 CSV 或 JSONL 格式。

JSONL

对于图像分割，JSONL索引文件有以下要求：

- 每个数据项都是一个单独的JSON对象，位于单独的行上。
- 键/值对`image_gcs_uri`是图像在Cloud Storage中的路径。
- 键/值对`category_mask_uri`是以PNG格式存储的掩模图像在Cloud Storage中的路径。
- 键/值对`'annotation_spec_colors'`是一个列表，将掩模颜色映射到标签。
  - 键/值对`display_name`是像素颜色掩模的标签。
  - 键/值对`color`是对应标签掩模的RGB归一化像素值（介于0和1之间）。

{ 'image_gcs_uri': image, 'segmentation_annotations': { 'category_mask_uri': mask_image, 'annotation_spec_colors' : [ { 'display_name': label, 'color': {"red": value, "blue", value, "green": value} }, ...] }

*注意*：字典键字段也可以采用驼峰命名法。例如，'image_gcs_uri'也可以是'imageGcsUri'。

云存储训练数据的位置。

现在将变量 `IMPORT_FILE` 设置为存储在云存储中的JSONL索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://ucaip-test-us-central1/dataset/isg_data.jsonl"

#### 快速查看您的数据

您将使用存储在公共云存储桶中的未知数据集的版本，使用一个JSONL索引文件。

首先快速查看数据。通过计算JSONL索引文件中对象的数量（`wc -l`）来统计示例的数量，然后查看前几行数据。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 导入数据

现在，将数据导入到您的 Vertex 数据集资源中。使用这个辅助函数 `import_data` 来导入数据。该函数执行以下操作：

- 使用 `Dataset` 客户端。
- 调用客户端的方法 `import_data`，并传入以下参数：
  - `name`：您给 `Dataset` 资源的人类可读名称（例如，unknown）。
  - `import_configs`：导入配置。

- `import_configs`：一个包含字典的Python列表，该字典包含以下键/值条目：
  - `gcs_sources`：指向一个或多个索引文件路径的URI列表。
  - `import_schema_uri`：指定标签类型的模式。

`import_data()` 方法将返回一个长时间运行的 `operation` 对象。这将需要几分钟的时间来完成。如果您正在进行实时教程，这将是询问问题或休息的好时机。

In [ ]:
def import_data(dataset, gcs_sources, schema):
    config = [{"gcs_source": {"uris": gcs_sources}, "import_schema_uri": schema}]
    print("dataset:", dataset_id)
    start_time = time.time()
    try:
        operation = clients["dataset"].import_data(
            name=dataset_id, import_configs=config
        )
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print(
            "after: running:",
            operation.running(),
            "done:",
            operation.done(),
            "cancelled:",
            operation.cancelled(),
        )

        return operation
    except Exception as e:
        print("exception:", e)
        return None


import_data(dataset_id, [IMPORT_FILE], LABEL_SCHEMA)

## 训练模型

现在使用你的 Vertex `Dataset` 资源来训练一个 AutoML 图像分割模型。请按照以下步骤训练模型：

1. 为该 `Dataset` 资源创建一个 Vertex 训练流水线。
2. 执行该流水线以开始训练。

创建一个训练管道

您可能会问，我们用管道做什么？通常，当工作（如训练）有多个步骤时，我们通常按顺序：执行步骤A，执行步骤B，等等。将这些步骤放入管道中，我们可以获得以下好处：

1. 对于后续训练作业可重复使用。
2. 可以容器化并作为批处理作业运行。
3. 可以分布式处理。
4. 所有步骤都与同一管道作业相关联，以便跟踪进度。

使用这个辅助函数 `create_pipeline`，它接受以下参数：

- `pipeline_name`：管道作业的人类可读名称。
- `model_name`：模型的人类可读名称。
- `dataset`：Vertex 完全合格的数据集标识符。
- `schema`：数据集标签（注释）训练模式。
- `task`：描述训练作业要求的字典。

该辅助函数调用 `Pipeline` 客户端服务的方法 `create_pipeline`，它接受以下参数：

- `parent`：您的 `Dataset`、`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
- `training_pipeline`：管道训练作业的完整规范。

现在让我们更深入地看一下构建 `training_pipeline` 规范所需的*最低*要求：

- `display_name`：管道作业的人类可读名称。
- `training_task_definition`：数据集标签（注释）训练模式。
- `training_task_inputs`：描述训练作业要求的字典。
- `model_to_upload`：模型的人类可读名称。
- `input_data_config`：数据集规范。
 - `dataset_id`：仅 Vertex 数据集标识符（非完全合格）--这是完全合格标识符的最后部分。
 - `fraction_split`：如果指定，数据集用于训练、测试和验证的百分比。否则，百分比将由 AutoML 自动选择。

In [ ]:
def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split("/")[-1]

    input_config = {
        "dataset_id": dataset_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    }

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients["pipeline"].create_training_pipeline(
            parent=PARENT, training_pipeline=training_pipeline
        )
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务要求

接下来，构建任务要求。与其他接受Python（类似JSON）字典的参数不同，`task`字段接受Google protobuf结构体，这与Python字典非常相似。使用`json_format.ParseDict`方法进行转换。

您需要指定的最小字段是：

- `budget_milli_node_hours`：用于训练模型的最大预算时间（计费），其中1000 = 1小时。
- `model_type`：部署模型的类型：
  - `CLOUD_HIGH_ACCURACY_1`：用于部署到Google Cloud并优化准确性。
  - `CLOUD_LOW_LATENCY_1`：用于部署到Google Cloud并优化延迟（响应时间）。

最后，通过调用辅助函数`create_pipeline`创建流水线，该函数将返回一个训练流水线对象的实例。

In [ ]:
PIPE_NAME = "unknown_pipe-" + TIMESTAMP
MODEL_NAME = "unknown_model-" + TIMESTAMP

task = json_format.ParseDict(
    {"budget_milli_node_hours": 2000, "model_type": "CLOUD_LOW_ACCURACY_1"}, Value()
)

response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在保存您创建的训练流水线的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split("/")[-1]

print(pipeline_id)

### 获取有关培训流水线的信息

现在只获取此培训流水线实例的流水线信息。通过调用作业客户端服务的 `get_training_pipeline` 方法，使用以下参数获取仅此作业的作业信息：

- `name`：Vertex 完全合格的流水线标识符。

当模型训练完成时，流水线状态将为 `PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients["pipeline"].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

# 部署

训练以上模型可能需要长达30分钟的时间。

一旦您的模型训练完成，您可以通过将`end_time`减去`start_time`来计算训练模型所需的实际时间。对于您的模型，您需要知道完全合格的Vertex模型资源标识符，即管道服务分配给它的。您可以从返回的管道实例中获取此信息，作为字段`model_to_deploy.name`。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

模型信息

现在你的模型已经训练好了，你可以获取一些关于你的模型的信息。

## 评估模型资源

现在找出模型服务认为你的模型有多好。作为训练的一部分，数据集的一部分被设置为测试（保留）数据，由管道服务用来评估模型。

### 列出所有切片的评估

请使用这个辅助函数`list_model_evaluations`，它需要以下参数：

- `name`：`Model`资源的Vertex完全合格的模型标识符。

这个辅助函数使用模型客户端服务的`list_model_evaluations`方法，该方法接受相同的参数。调用的响应对象是一个列表，其中每个元素都是评估指标。

对于每个评估--您可能只有一个，然后我们打印出每个评估中每个指标的所有键名称，对于一个小集合(`confidenceMetricsEntries`)，您将打印结果。

In [ ]:
def list_model_evaluations(name):
    response = clients["model"].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print("confidenceMetricsEntries", metrics["confidenceMetricsEntries"])

    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

部署`Model`资源

现在部署您使用AutoML创建的训练后的Vertex` Model`资源。 这需要两个步骤：

1. 为`Model`资源创建一个用于部署的`Endpoint`资源。

2. 将`Model`资源部署到`Endpoint`资源中。

### 创建一个`Endpoint`资源

使用这个辅助函数`create_endpoint`来创建一个端点，用于部署模型以提供预测，参数如下：

- `display_name`：`Endpoint`资源的可读名称。

辅助函数使用端点客户端服务的`create_endpoint`方法，该方法接受以下参数：

- `display_name`：`Endpoint`资源的可读名称。

创建一个`Endpoint`资源会返回一个长时间运行的操作，因为为了提供服务可能需要一些时间来配置`Endpoint`资源。您可以调用`response.result()`，这是一个同步调用，当Endpoint资源准备就绪时会返回。辅助函数返回`Endpoint`资源的Vertex全限定标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "unknown_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的“Endpoint”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例扩展

处理在线预测请求时，您有几种计算实例扩展选择：

- 单一实例：在线预测请求在单个计算实例上处理。
   -将计算实例的最小 (`MIN_NODES`) 和最大 (`MAX_NODES`) 数量设置为一。

- 手动扩展：在线预测请求在您手动指定的固定数量的计算实例上分配。
   -将计算实例的最小 (`MIN_NODES`) 和最大 (`MAX_NODES`) 数量设置为相同数量的节点。当模型首次部署到实例时，固定数量的计算实例将被配置，并且在线预测请求将在它们之间均匀分配。

- 自动扩展：在线预测请求在可扩展数量的计算实例上分配。
   -将计算实例的最小 (`MIN_NODES`) 数量设置为首次部署模型时要配置和解除配置的实例数量，并将最大 (`MAX_NODES) 数量设置为根据负载条件要配置的实例数量。

计算实例的最小数量对应于字段 `min_replica_count`，最大数量对应于字段 `max_replica_count`，在您之后的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署 `Model` 资源到 `Endpoint` 资源

使用这个辅助函数 `deploy_model` 来将您为提供预测创建的 `Model` 资源部署到 `Endpoint` 资源，使用以下参数：

- `model`：要从训练管道上传（部署）的 Vertex 完全合格的模型标识符。
- `deploy_model_display_name`：部署模型的人类可读名称。
- `endpoint`：要部署模型的 Vertex 完全合格的终端点标识符。

辅助函数调用 `Endpoint` 客户端服务的 `deploy_model` 方法，该方法接受以下参数：

- `endpoint`：要部署 `Model` 资源到的 Vertex 完全合格的 `Endpoint` 资源标识符。
- `deployed_model`：部署模型的要求规范。
- `traffic_split`：端点中流量的百分比将发送到此模型，指定为一个或多个键/值对的字典。
   - 如果只有一个模型，则指定为 **{ "0": 100 }**，其中 "0" 指代要上传的这个模型，100 表示全部流量的 100%。
   - 如果终端上有现有模型，需要分担流量，则使用 `model_id` 指定为 **{ "0": percent, model_id: percent, ... }**，其中 `model_id` 是部署到终端的现有模型的模型标识符。百分比必须加起来等于 100。

现在让我们更深入地了解 `deployed_model` 参数。此参数作为一个具有最小必需字段的 Python 字典指定：

- `model`：要部署的（上传）模型的 Vertex 完全合格的模型标识符。
- `display_name`：部署模型的人类可读名称。
- `disable_container_logging`：这将禁用容器事件的日志记录，例如执行失败（默认情况下启用容器日志记录）。在调试部署时通常启用容器日志记录，然后在为生产部署后禁用。
- `automatic_resources`：这指的是多少个冗余计算实例（副本）。在这个例子中，我们将其设置为一个（不复制）。

#### 流量分割

现在让我们更深入地了解 `traffic_split` 参数。此参数作为一个Python字典指定。这一开始可能有点困惑。让我解释一下，您可以将多个模型实例部署到一个端点中，然后设置每个实例获得多少（百分之几）的流量。

为什么要这样做？也许您已经在生产环境中部署了之前的版本 -- 让我们称之为 v1。您在 v2 上获得了更好的模型评估，但在部署到生产环境之前，您无法确定它是否真的更好。因此，在流量分割的情况下，您可能希望将 v2 部署到与 v1 相同的端点，但只获得 10% 的流量。这样，您可以在不干扰大多数用户的情况下监视其表现，直到做出最终决定。

#### 响应

该方法返回一个长时间运行的操作 `response`。我们将同步等待操作完成，调用 `response.result()`，这将阻塞直到模型部署完成。如果这是第一次将模型部署到端点，则可能需要额外几分钟来完成资源的分配。

In [ ]:
DEPLOYED_NAME = "unknown_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "automatic_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
        },
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

## 发起一个在线预测请求

现在对您部署的模型进行在线预测。

### 获取测试项

您将从数据集中随机选择一个示例作为测试项。不必担心该示例很可能在训练模型时已经使用过 — 我们只是想展示如何进行预测。

In [ ]:
import json

test_items = !gsutil cat $IMPORT_FILE | head -n1
test_data = test_items[0].replace("'", '"')
test_data = json.loads(test_data)
try:
    test_item = test_data["image_gcs_uri"]
    test_label = test_data["segmentation_annotation"]["annotation_spec_colors"]
except:
    test_item = test_data["imageGcsUri"]
    test_label = test_data["segmentationAnnotation"]["annotationSpecColors"]

print(test_item, test_label)

###做出预测

现在您有一个测试项。使用这个辅助函数`predict_item`，它接受以下参数：

- `filename`：测试项在Cloud Storage中的路径。
- `endpoint`：`Model`资源部署的Vertex的完全合格标识符中的`Endpoint`资源。
- `parameters_dict`：用于提供预测结果的额外过滤参数。

此函数调用预测客户端服务的`predict`方法，并使用以下参数：

- `endpoint`：`Model`资源部署的Vertex的完全合格标识符中的`Endpoint`资源。
- `instances`：要预测的实例（编码图像）列表。
- `parameters`：用于提供预测结果的额外过滤参数。 *注意*，图像分割模型不支持额外的参数。

####请求

由于在本示例中您的测试项位于Cloud Storage存储桶中，您将使用`tf.io.gfile.Gfile()`打开并读取图像的内容。为了将测试数据传递给预测服务，我们将将字节编码为base64 -- 这样二进制数据在通过互联网传输时就不会被修改。

每个实例的格式为：

     { 'content': { 'b64': [base64编码的字节] } }

由于`predict()`方法可以接受多个项（实例），您可以将我们的单个测试项作为一个测试项列表发送。最后一步，您将实例列表打包成Google的protobuf格式 -- 这是我们传递给`predict()`方法的内容。

####响应

`response`对象返回一个列表，列表中的每个元素对应请求中的相应图像。您将在每个预测的输出中看到以下内容 -- 在我们的情况下只有一个：

- `confidenceMask`：预测的置信水平。
- `categoryMask`：每个像素的预测标签。

In [ ]:
import base64

import tensorflow as tf


def predict_item(filename, endpoint, parameters_dict):

    parameters = json_format.ParseDict(parameters_dict, Value())
    with tf.io.gfile.GFile(filename, "rb") as f:
        content = f.read()
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{"content": base64.b64encode(content).decode("utf-8")}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", dict(prediction))


predict_item(test_item, endpoint_id, None)

取消部署`Model`资源

现在从服务`Endpoint`资源中取消部署您的`Model`资源。使用这个辅助函数`undeploy_model`，它接受以下参数：

- `deployed_model_id`：当`Model`资源部署到时由端点服务返回的模型部署标识符。
- `endpoint`：`Model`部署到的`Endpoint`资源的Vertex完全限定标识符。

该函数调用端点客户端服务的方法`undeploy_model`，并传入以下参数：

- `deployed_model_id`：当`Model`资源被部署时由端点服务返回的模型部署标识符。
- `endpoint`：`Model`资源部署到的`Endpoint`资源的Vertex完全限定标识符。
- `traffic_split`：如何在`Endpoint`资源上分配流量到其余部署的模型。

由于这是`Endpoint`资源上唯一部署的模型，您可以通过将其设置为空对象{}来简单地将`traffic_split`留空。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理

要清理此项目中使用的所有GCP资源，您可以删除用于本教程的GCP项目。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME